# Imports

Agrega librerias para la fila y creacion de copias de objetos

In [ ]:
import queue
import copy

# Clase para representar soluciones

Un objeto de la clase `Partial_Sol` incluye:  
* la lista de objetos elegidos
* el indice de la siguiente variable a instancias

Para el problema knapsack:
* valor acumulado
* peso acumulado
* upperbound potencial


In [ ]:
class Partial_Sol:

	def __init__(self, lista, upperbound):
		self.lista = lista
		self.indef_index = 0

		self.valor = 0
		self.peso = 0
		self.ub = upperbound

In [ ]:
def extend(sol_padre, v, w):

	# Crea soluciones hijas como copias de la solucion padre
	hija1 = copy.deepcopy(sol_padre)
	hija2 = copy.deepcopy(sol_padre)

	# Extiende las soluciones hijas instanciando su siguiente variable indefinida
  # Luego actualiza el indef_index de las hijas
	indice = sol_padre.indef_index
	hija1.lista[indice] = 1
	hija2.lista[indice] = 0

	hija1.indef_index += 1
	hija2.indef_index += 1
	# Actualiza tambien el valor y peso de las sol hija
	hija1.valor += v[indice]
	hija1.peso += w[indice]

  # Devuelve las hijas como una lista
	return [hija1, hija2]


# Evaluacion del upperbound

Determina `best_ratio`: el mejor ratio de entre los objetos que aun estan indefinidos.

Estimado optimistamente, cada una de las unidades de peso pendientes de asignar, podria ser ocupada por el mejor ratio



In [ ]:
# Recibe como parametros a la solucion, y las variables que representan al problema
def evaluate(hija, K, R):

	# Encuentra el mejor ratio (el mayor) de entre los objetos por decidir (los que todavia son -1)
	indice = hija.indef_index
	best_ratio = max(R[indice:])

  # El upperbound ub es el valor actual + peso restante * mejor ratio potencial
	ub = hija.valor + (K - hija.peso) * best_ratio

  # Devuelve el upperbound calculado
	return ub

# El problema

Define el problema:
* vector de valores `V`,
* pesos `W`,
* la capacidad `K`,
* numero de objetos `n`
* vector de ratios de valor/peso `R`

In [ ]:

#	Knapsack
#   Valores V, pesos W, capacidad K y ratios R
#   Numero de objetos n
V = [40, 42, 25, 12 ]
W = [ 4,  7,  5,  3 ]
K = 10
n = len(V)

R = [V[i]/W[i] for i in range(n)]

# Solucion inicial

In [ ]:

# Crea un objeto de la clase de la solucion inicial
lista = [-1] * n
best_ratio = max(R)

root_sol = Partial_Sol(lista, best_ratio * K)

# Crea e inicializa la fila de prioridad
# La fila de prioridad requiere un par ordenado de prioridad y objeto
q = queue.PriorityQueue()
q.put( (0, root_sol )   )

# best_so_far guardara la mejor solucion completa encontrada hasta el momento
best_so_far = copy.deepcopy( root_sol )


# El ciclo principal

Mientras la fila de prioridad no este vacia:
  * Extrae una `solucion padre`
  * Si su estimacion de costo mas optimista (el `upperbound`) es peor que el costo real de la solucion `best_so_far`, se ignora
  * Si no, la solucion padre tiene potencial. Crea a sus `hijas`.
  * Por cada hija:
    * Revisa si es una solucion completamente definida, mejor que `best_so_far`. De ser asi, actualizala
    * Si no esta completamente definida, pero es valida y tiene potencial de mejorar a best_so_far, ingresa en la fila, con `-h.ub` como prioridad

In [ ]:
while not q.empty():
  # Extrae una solucion padre de la fila
  # Usa q.get()[1] para obtener el segundo elemento del par ordenado
	partial_sol = q.get()[1]

  # Si la solucion padre tiene un upperbound peor que el valor de best_so_far, se ignora
	if partial_sol.ub <= best_so_far.valor:
		print("Ignorada con valor ", partial_sol.valor, " y upperbound ", partial_sol.ub, " por que best so far ", best_so_far.valor)

	# Si no, crea a sus hijas
	else:
		hijas = extend(partial_sol, V, W)

		#	Por cada hija
		for h in hijas:
			# Evalua y asigna su upperbound
			h.ub = evaluate(h, K, R)

      # Revisa si la sol hija esta completa, y ademas es valida y mejor que best_so_far
      # En ese caso, actualiza best_so_far con la sol hija
			if h.indef_index == n and h.peso <= K and h.valor > best_so_far.valor:
				best_so_far = h

			# Si la hija esta incompleta, es valida y su upperbound tiene el potencial
      # de mejorar a best_so_far, ingresa en la fila, con -upperbound como prioridad
			if h.indef_index < n and h.ub > best_so_far.valor and h.peso <= K:
				q.put((-h.ub, h))


ValueError: max() iterable argument is empty

# Muestra el resultado de `best_so_far`

In [ ]:
print("\nLa mejor sol: ")
print(best_so_far.lista, best_so_far.valor, best_so_far.peso, best_so_far.ub)